In [3]:
from tqdm import tqdm

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST

# Extracting Flattened Features

In [4]:
train_data = FashionMNIST(root='./Data', download=True, train=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=0.2860, std=0.3530)]))
test_data = FashionMNIST(root='./Data', download=True, train=False, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=0.2860, std=0.3530)]))

train_data, val_data = random_split(
    train_data, [55000, 5000], generator=torch.Generator().manual_seed(10))

data = {'train': train_data,
        'val': val_data,
        'test': test_data}

In [5]:
# feature size should be nx(28*28)

save_path = './Data/Features/flattened_features.pt'
batch_size = 128

features = {}
for key, value in data.items():
    print(f'\nExtracting {key} Features...')

    loader = DataLoader(value, batch_size, shuffle=False, num_workers=3)

    running_features, running_labels = torch.tensor([]), torch.tensor([])
    for images, labels in tqdm(loader):
        running_features = torch.cat(
            [running_features, torch.flatten(images, start_dim=1)], dim=0)
        running_labels = torch.cat([running_labels, labels], dim=0)

    features[key] = [running_features, running_labels]

torch.save(features, save_path)
print("\nFeature Dict Saved to, ", save_path)


Extracting train Features...


100%|██████████| 430/430 [00:09<00:00, 44.52it/s]



Extracting val Features...


100%|██████████| 40/40 [00:00<00:00, 64.13it/s]



Extracting test Features...


100%|██████████| 79/79 [00:00<00:00, 82.13it/s]



Feature Dict Saved to,  ./Data/Features/flattened_features.pt


# Extracting Resnet18 Features

In [6]:
from torchvision.models import ResNet18_Weights

train_data = FashionMNIST(root='./Data', download=True, train=True, transform=transforms.Compose(
    [transforms.Lambda(lambda x: x.convert('RGB')), ResNet18_Weights.IMAGENET1K_V1.transforms(antialias=True)]))
test_data = FashionMNIST(root='./Data', download=True, train=False, transform=transforms.Compose(
    [transforms.Lambda(lambda x: x.convert('RGB')), ResNet18_Weights.IMAGENET1K_V1.transforms(antialias=True)]))

train_data, val_data = random_split(
    train_data, [55000, 5000], generator=torch.Generator().manual_seed(10))

data = {'train': train_data,
        'val': val_data,
        'test': test_data}

In [7]:
from torchvision.models import resnet18

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18()
# Weights from https://download.pytorch.org/models/resnet18-f37072fd.pth
model.load_state_dict(torch.load('./Data/Models/resnet18-f37072fd.pth'))

model.eval()
model.to(device)

In [8]:
# feature size should be nx1000

save_path = './Data/Features/resnet18_features.pt'
batch_size = 128
num_workers = 5

features = {}
for key, value in data.items():
    print(f'\nExtracting {key} Features...')

    loader = DataLoader(value, batch_size, num_workers=num_workers)

    running_features, running_labels = torch.tensor([]), torch.tensor([])
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            running_features = torch.cat(
                [running_features, model(images).to('cpu')], dim=0)
            running_labels = torch.cat([running_labels, labels], dim=0)

    features[key] = [running_features, running_labels]

torch.save(features, save_path)
print("\nFeature Dict Saved to, ", save_path)


Extracting train Features...


100%|██████████| 430/430 [01:35<00:00,  4.51it/s]



Extracting val Features...


100%|██████████| 40/40 [00:08<00:00,  4.64it/s]



Extracting test Features...


100%|██████████| 79/79 [00:16<00:00,  4.68it/s]



Feature Dict Saved to,  ./Data/Features/resnet18_features.pt


# Extracting Vit Features

In [9]:
from torchvision.models import ViT_B_16_Weights

train_data = FashionMNIST(root='./Data', download=True, train=True, transform=transforms.Compose(
    [transforms.Lambda(lambda x: x.convert('RGB')), ViT_B_16_Weights.IMAGENET1K_V1.transforms(antialias=True)]))
test_data = FashionMNIST(root='./Data', download=True, train=False, transform=transforms.Compose(
    [transforms.Lambda(lambda x: x.convert('RGB')), ViT_B_16_Weights.IMAGENET1K_V1.transforms(antialias=True)]))

train_data, val_data = random_split(
    train_data, [55000, 5000], generator=torch.Generator().manual_seed(10))

data = {'train': train_data,
        'val': val_data,
        'test': test_data}

In [10]:
from torchvision.models import vit_b_16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = vit_b_16()
# Weights from https://download.pytorch.org/models/vit_b_16-c867db91.pth
model.load_state_dict(torch.load('./Data/Models/vit_b_16-c867db91.pth'))

model.eval()
model.to(device)

In [11]:
# feature size should be nx1000

save_path = './Data/Features/vit_b_16_features.pt'
batch_size = 128
num_workers = 5

features = {}
for key, value in data.items():
    print(f'\nExtracting {key} Features...')

    loader = DataLoader(value, batch_size, num_workers=num_workers)

    running_features, running_labels = torch.tensor([]), torch.tensor([])
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            running_features = torch.cat(
                [running_features, model(images).to('cpu')], dim=0)
            running_labels = torch.cat([running_labels, labels], dim=0)

    features[key] = [running_features, running_labels]

torch.save(features, save_path)
print("\nFeature Dict Saved to, ", save_path)


Extracting train Features...


100%|██████████| 430/430 [23:58<00:00,  3.35s/it]



Extracting val Features...


100%|██████████| 40/40 [02:09<00:00,  3.25s/it]



Extracting test Features...


100%|██████████| 79/79 [04:19<00:00,  3.28s/it]



Feature Dict Saved to,  ./Data/Features/vit_b_16_features.pt
